In [1]:
import os
from IPython.display import Image, display
import random
import pandas as pd
import numpy as np

import sys
sys.path.append('../d03_src/')
import utils_data as prd
import utils_evaluation as ev
import utils_visualization as vis
from utils import add_trivial_inspections, set_figsize, latex
import vars

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Helvetica Light' #comment in case not installed
plt.rc('axes', unicode_minus=False)

np.random.seed(13)

ModuleNotFoundError: No module named 'utils_data'

In [5]:
import os

dir = '/share/garg/gs665/DEPRECATED_networks_underreporting/d05_joboutputs/calibration/'
reg = [int(f.split('.')[0].split('-')[1]) for f in os.listdir(f'{dir}regression-psi-all6/nodestats/') if 'csv' in f]
homo = [int(f.split('.')[0].split('-')[1]) for f in os.listdir(f'{dir}homogeneous-psi/nodestats/') if 'csv' in f]
homo_reg = [int(f.split('.')[0].split('-')[1]) for f in os.listdir(f'{dir}homogeneous-psi-regression-data/nodestats/') if 'csv' in f]

reg.sort()
homo.sort()
homo_reg.sort()

common_indices = list(set(homo)&set(reg)&set(homo_reg))
print(f'Available number of experiments:', len(common_indices))

nearest_hundred = len(common_indices) - len(common_indices)%100
print(f'Will round it to {nearest_hundred} experiments')
indices_to_use = common_indices[:nearest_hundred]

Available number of experiments: 591
Will round it to 500 experiments


In [4]:
for x in common_indices:
    for dir in ['homogeneous-psi', 'homogeneous-psi-regression-data', 'regression-psi-all6']:
        print(f'cp {dir}/)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186

Parameters to be set:

In [ ]:
calibration_job_dir = "../d05_joboutputs/calibration/"
n_experiments_to_display=5

# Evaluating Calibration on Synthetic Data

In this notebook we go through the calibration for the real NY graph with $N=2255$ nodes

## Parameters

### Chain Parameters

I am running $2$ chains for $N=60,000$ iterations. The first $20,000$ are discarded, and the remaining $40,000$ are thinned with probability $0.5$

In [ ]:
n_chains = 2
n_iter = 60_000
n_burnin = 20_000

The $\vec{\theta}$ parameters are sampled using the Single-Variable Exchange Algorithm, with Swandseng-Wang sampling to generate an auxiliary variable from the Ising Model. Each auxiliary variable is generated after $50$ iterations. The ground truth $\vec{A}$ is sampled on a Gibbs routine given $\vec{T}$ and all other parameters. 

### Priors

The prior I am using on $\theta$ is

$$ \theta_0 \sim \mathcal{N}\left(0.0, 0.50\right)$$
$$ \theta_1 \sim \mathcal{N}\left(0.1, 0.03\right)$$

For the case where the report rate $\psi$ is homogenous, I am using a Beta prior:

$$\psi \sim \text{Beta}\left(\text{mean}=0.6, \text{strength}=2\right)$$

For the case where the report rate $\psi$ is dependent on demographics, I am using the following covariates:

\begin{align*}
X_1 &:= \log(\text{population})\\
X_2 &:= \text{median income}\\
X_3 &:= \text{population with at least a bachelor degree }(\%)\\
X_4 &:= \text{non-hispanic population identifying as white alone }(\%)\\
X_5 &:= \text{median age}\\
X_6 &:= \text{households occupied by renter }(\%)
\end{align*}

All covariates are standardized. The reporting rate of cbg $i$ is a logistic linear combination of the form:

$$ \psi_i = \text{logit}^{-1}\left(\alpha_0 + \sum_{k=1}^6\alpha_k\cdot X_{ki}\right)$$

The priors used for the regression coefficients are:

$$\alpha_0 \sim \mathcal{N}\left(0, 1.0\right)$$
$$\alpha_1,\dots \alpha_6 \sim \mathcal{N}\left(0, 0.5\right)$$

### Graph

The graph is that of census tracts, with nodes corresponding to zero population nodes and very long edges removed

In [ ]:
ny_gdf, ny_graph, base_gdf = prd.generate_graph_census(census_unit='tracts',
                                                       remove_high_degree_nodes=False,
                                                       remove_long_edges=True,
                                                       remove_parks=False,
                                                       remove_zeropop=True)

ny_gdf = ny_gdf.to_crs(vars._projected_crs)
base_gdf = base_gdf.to_crs(vars._projected_crs)
_ = prd.draw_gdf_and_graph(ny_gdf, ny_graph, base_gdf)

N = ny_graph.order()
print(f'There are {N} nodes in the graph')

In [ ]:
plots_dict = {}
summaries_dict = {}
nodestats_dict = {}

converged_plots_dict = {}
converged_summaries_dict = {}
converged_nodestats_dict = {}

## 1. Homogeneous Psi

In [ ]:
model = 'homogeneous-psi'
model_params = ['theta0', 'theta1', 'psi']

In [ ]:
plots_dict[model], summaries_dict[model], nodestats_dict[model] = ev.get_summary_files(model, calibration_job_dir,
                                                                                       ny_gdf=ny_gdf,
                                                                                       include_demographics=True,
                                                                                       mode='calibration')
#Include baseline:
nodestats_dict[f'Baseline Model ({model})'] = [add_trivial_inspections(nodestats_df.copy(), ny_graph).assign(inf_psi=1) for nodestats_df in nodestats_dict[model]]
for nodestats_df in nodestats_dict[f'Baseline Model ({model})']: nodestats_df['inf_Tprob'] = nodestats_df['inf_Aprob']

Convergence diagnostics:

In [ ]:
rhats = np.array([list(summary.r_hats.values)[:-1] for summary in summaries_dict[model]])
converged_rows = list(np.nonzero((rhats < 1.1).sum(axis=1) == len(model_params))[0])

converged_plots_dict[model] = [plots_dict[model][i] for i in converged_rows]
converged_summaries_dict[model] = [summaries_dict[model][i] for i in converged_rows]
converged_nodestats_dict[model] = [nodestats_dict[model][i] for i in converged_rows]

fig, ax = plt.subplots(figsize=(10, 5))
_ = ax.hist(rhats, label=model_params)
_ = ax.set_title(r'$\hat{R}$ histogram for model parameters')
_ = ax.legend()

print(f'We have {len(summaries_dict[model])} experiments, out of which {len(converged_summaries_dict[model])} ({len(converged_summaries_dict[model])/len(summaries_dict[model]):.2%} %) converged.')

We have the data summaries saved, as well as chain traces and posterior plots. Let's show how a few experiments went:

In [ ]:
available_plots = plots_dict[model]
random.shuffle(available_plots)
for plot in available_plots[:n_experiments_to_display]:
    display(plot)

The calibration plots:

In [ ]:
df_dict = ev.evaluate_summaries(summaries_dict[model], param_names=model_params+['A_mean'])
fig, Axes = plt.subplots(figsize=(12, 12), ncols=2, nrows=2)
for param, ax in  zip(model_params+['A_mean'], Axes.flatten()):
    _ = vis.plot_evaluation_scatterplot(df_dict[param], title=latex(param), ax=ax, show=False)
    

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
_ = vis.plot_calibration(summaries_dict[model],
                         param_names=model_params,
                         title_size=15,
                         ticks_size=13,
                         xlabel_size=14,
                         ylabel_size=14,
                         leg_size=13,
                         leg_ncols=1,
                         title='Homogeneous Reporting Model Calibration\n',
                         ax=ax,show=False)
#fig.savefig('../_paper-plots/calibration_homogeneous.pdf', bbox_inches='tight')

In [ ]:
AUC_list = ev.compute_AUC(nodestats_dict[model],
                          real_data=False, use_Aprob=True,
                          average_AUCs=False, plot=False, show=True)
print('For the under-reporting model')
print(f'The AUC is, on average, {np.mean(AUC_list):.2f} with 95% CI ({np.quantile(AUC_list, 0.275):.2f},{np.quantile(AUC_list, 0.975):.2f})')

base_AUC_list = ev.compute_AUC(nodestats_dict[f'Baseline Model ({model})'],
                               real_data=False, use_Aprob=True,
                               average_AUCs=False,
                               plot=False, show=True)
print('\nFor the baseline model')
print(f'The AUC is, on average, {np.mean(base_AUC_list):.2f} with 95% CI ({np.quantile(base_AUC_list, 0.275):.2f},{np.quantile(base_AUC_list, 0.975):.2f})')

## Regression

In [ ]:
model = 'regression-psi-all6'
model_params = ['theta0', 'theta1'] + [f'alpha{k}' for k in range(7)]

In [ ]:
plots_dict[model], summaries_dict[model], nodestats_dict[model] = ev.get_summary_files(model, calibration_job_dir,
                                                                                       ny_gdf=ny_gdf,
                                                                                       include_demographics=True,
                                                                                       mode='calibration')
#Include baseline:
nodestats_dict[f'Baseline Model ({model})'] = [add_trivial_inspections(nodestats_df.copy(), ny_graph).assign(inf_psi=1) for nodestats_df in nodestats_dict[model]]
for nodestats_df in nodestats_dict[f'Baseline Model ({model})']: nodestats_df['inf_Tprob'] = nodestats_df['inf_Aprob']

Convergence diagnostics:

In [ ]:
rhats = np.array([list(summary.r_hats.values)[:-1] for summary in summaries_dict[model]])
converged_rows = list(np.nonzero((rhats < 1.1).sum(axis=1) == len(model_params))[0])

converged_plots_dict[model] = [plots_dict[model][i] for i in converged_rows]
converged_summaries_dict[model] = [summaries_dict[model][i] for i in converged_rows]
converged_nodestats_dict[model] = [nodestats_dict[model][i] for i in converged_rows]

fig, ax = plt.subplots(figsize=(10, 5))
_ = ax.hist(rhats, label=model_params)
_ = ax.set_title(r'$\hat{R}$ histogram for model parameters')
_ = ax.legend()

print(f'We have {len(summaries_dict[model])} experiments, out of which {len(converged_summaries_dict[model])} ({len(converged_summaries_dict[model])/len(summaries_dict[model]):.2%} %) converged.')

We have the data summaries saved, as well as chain traces and posterior plots. Let's show how a few experiments went:

In [ ]:
available_plots = plots_dict[model]
random.shuffle(available_plots)
for plot in available_plots[:n_experiments_to_display]:
    display(plot)

The calibration plots:

In [ ]:
df_dict = ev.evaluate_summaries(summaries_dict[model], param_names=model_params+['A_mean'])
fig, Axes = plt.subplots(figsize=(18, 18), ncols=3, nrows=3)
for idx, (param, ax) in  enumerate(zip(model_params+['A_mean'], Axes.flatten())):
    _ = vis.plot_evaluation_scatterplot(df_dict[param],
                                        title=latex(param),
                                        ax=ax, show=False,
                                        ylabel='Posterior mean' if idx==3 else None,
                                        xlabel='True parameter' if idx==7 else None)

plt.savefig('../_paper-plots/calibration_heterogeneous_scatterplot_full.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
_ = vis.plot_calibration(summaries_dict[model],
                         param_names=model_params,
                         title_size=15,
                         ticks_size=13,
                         xlabel_size=14,
                         ylabel_size=14,
                         leg_size=13,
                         leg_ncols=1,
                         title='Heterogeneous Reporting Model Calibration\n',
                         ax=ax,show=False)
plt.savefig('../_paper-plots/calibration_heterogeneous.pdf', bbox_inches='tight')

In [ ]:
fig, Axes = plt.subplots(figsize=(12,6), ncols=2)

_ = vis.plot_calibration(summaries_dict['homogeneous-psi'],
                         param_names=['theta0', 'theta1', 'psi'],
                         title_size=15,
                         ticks_size=13,
                         xlabel_size=14,
                         xlabel=None,
                         ylabel_size=14,
                         leg_size=13,
                         leg_ncols=1,
                         title='Homogeneous Reporting Model\n',
                         ax=Axes[0],show=False)

_ = vis.plot_calibration(summaries_dict[model],
                         param_names=model_params,
                         title_size=15,
                         ticks_size=13,
                         xlabel_size=14,
                         ylabel=None,
                         ylabel_size=14,
                         leg_size=13,
                         leg_ncols=1,
                         title='Heterogeneous Reporting Model\n',
                         ax=Axes[1],show=False)
plt.savefig('../_paper-plots/calibration_maintext.pdf', bbox_inches='tight')

For the paper we will put them together:

In [ ]:
fig, Axes = plt.subplots(figsize=(10, 10), ncols=2, nrows=2)

params_to_plot = ['theta0', 'alpha0', 'alpha2', 'alpha4']
params_to_plot_annot = ['(flood prevalence)', '(intercept)', '(median income)', '(white population)']
for idx, (param, ax) in enumerate(zip(params_to_plot, Axes.flatten())): #2: median income and 4: race white
    _ = vis.plot_evaluation_scatterplot(df_dict[param],
                                        title=latex(param) +' '+params_to_plot_annot[idx],
                                        ylabel = 'Posterior mean' if idx in [0,2] else None,
                                        xlabel = 'True parameter' if idx in [2,3] else None,
                                        correlation_size=10,
                                        ax=ax, show=False)
plt.savefig('../_paper-plots/calibration_heterogeneous_scatterplot_maintext.pdf', bbox_inches='tight')

AUC:

In [ ]:
AUC_list = ev.compute_AUC(nodestats_dict[model],
                          real_data=False, use_Aprob=True,
                          average_AUCs=False, plot=False, show=True)
print('For the heterogeneous reporting model')
print(f'The AUC is, on average, {np.mean(AUC_list):.2f} with 95% CI ({np.quantile(AUC_list, 0.275):.2f},{np.quantile(AUC_list, 0.975):.2f})')

base_AUC_list = ev.compute_AUC(nodestats_dict[f'Baseline Model ({model})'],
                               real_data=False, use_Aprob=True,
                               average_AUCs=False,
                               plot=False, show=True)
print('\nFor the baseline model')
print(f'The AUC is, on average, {np.mean(base_AUC_list):.2f} with 95% CI ({np.quantile(base_AUC_list, 0.275):.2f},{np.quantile(base_AUC_list, 0.975):.2f})')